# Fitting Thermoelectric data
Models and data are from Danny/Kedar.

## Import Modules, Functions, and Data

`functions.py` has the Python implementations of all the helper functions (I used a previously written package, `fdint`, for the Fermi-Dirac integrals).

In [1]:
import numpy as np
from functions import *

In [2]:
celldata = {}
celldata['xdata'] = np.loadtxt('xdata.csv',delimiter=',')
celldata['ydata'] = np.loadtxt('ydata.csv',delimiter=',')
celldata['n'] = 40

## Validate Python implementation
I did a test evaluation in Matlab and Python with the same input parameters. Let's import the results and compare to make sure we're getting the same thing.

In [3]:
test_in = [-2.499946233286e1,1.833885014595e-3,-2.2588468610036e-3,8.6217332036812e-4]
test_y,test_S,test_Rou=tefunnew(celldata,test_in)
matlab_y = np.loadtxt('matlab_y.csv',delimiter=',')
matlab_S = np.loadtxt('matlab_S.csv',delimiter=',')
matlab_Rou = np.loadtxt('matlab_Rou.csv',delimiter=',')

In [4]:
y_pct_diff=(test_y-matlab_y)/matlab_y
print('There is an average of a %.2f%% difference (with a standard deviation of %.2f%%) between the Matlab and Python implementations in the y output.'%(round(100.0*np.mean(y_pct_diff),2),round(100.0*np.std(y_pct_diff),2)))

There is an average of a 0.27% difference (with a standard deviation of 0.00%) between the Matlab and Python implementations in the y output.


In [5]:
S_pct_diff=(test_S-matlab_S)/matlab_S
print('There is an average of a %.2f%% difference (with a standard deviation of %.2f%%) between the Matlab and Python implementations in the S output.'%(round(100.0*np.mean(S_pct_diff),2),round(100.0*np.std(S_pct_diff),2)))

There is an average of a 0.13% difference (with a standard deviation of 0.00%) between the Matlab and Python implementations in the S output.


In [6]:
Rou_pct_diff=(test_Rou-matlab_Rou)/matlab_Rou
print('There is an average of a %.3f%% difference (with a standard deviation of %.3f%%) between the Matlab and Python implementations in the Rou output.'%(round(100.0*np.mean(Rou_pct_diff),3),round(100.0*np.std(Rou_pct_diff),3)))

There is an average of a -0.051% difference (with a standard deviation of 0.002%) between the Matlab and Python implementations in the Rou output.


Okay, so the differences aren't nothing, but they're small enough that I think we can work with them.

## Fitting with Bayesim
Now let's do a fit to the data using the grid approach implemented in the `bayesim` code.
### Import Things

In [7]:
import sys
sys.path.append('../../')
import bayesim.model as bym
import bayesim.param_list as byp
import functions as tefcns # model functions implemented in a separate file to keep this notebook tidy
import deepdish as dd # for interacting with HDF5 files
from joblib import Parallel, delayed # to parallelize model computations

### Initialize
First, we set up the list of parameters to be fit and their ranges.

In [8]:
fp = byp.param_list()
"""
fp.add_fit_param(name='P0', val_range=[1e-34,1e-20], spacing='log', length=28, units='sec.')
fp.add_fit_param(name='fs', val_range=[-1,2], length=21, units='eV')
fp.add_fit_param(name='r', val_range=[-1,2], length=21)
fp.add_fit_param(name='Z', val_range=[-10,10], length=20)
"""
fp.add_fit_param(name='P0', val_range=[1e-34,1e-20], spacing='log', length=7, units='sec.')
fp.add_fit_param(name='fs', val_range=[-1,2], length=5, units='eV')
fp.add_fit_param(name='r', val_range=[-1,2], length=5)
fp.add_fit_param(name='Z', val_range=[-10,10], length=5)


Next, define the experimental conditions.

In [9]:
ec = ['T','R','n']

Now, set up the `bayesim.model` object. All we need to feed in are the parameters, experimental conditions, and name of the output variable.

In [10]:
m = bym.model(params=fp,ec=ec,output_var='P')

### Attach Experimental Observations
The next thing to do is to attach the observed data. I reformatted it to work with `bayesim` and saved an HDF5 file. You can see the format in the Excel sheet `TE_expt_data.xlsx`. Here I use only every third point (integer values of resistances) to speed up model computation and also because that's probably enough data.

In [11]:
#m.attach_observations(fpath='TE_expt_data.h5')
m.attach_observations(fpath='TE_expt_data_sparse.h5')

Identified experimental conditions as ['n', 'T', 'R']. If this is wrong, rerun and explicitly specify them with attach_ec (make sure they match data file columns) or remove extra columns from data file.


### Attaching the Model
Next, we attach the model. In this example I'll precompute the modeled data and attach a file with the outputs. You could also attach the function used to do the modeling, but the code can't currently parallelize those computations so I do it outside `bayesim` to take advantage of both cores on my laptop.
First we write out a file with the list of all simulation points. (it's good practice to write this out rather than keep it only as a Python object so we can pick up where we left off later)

This next cell should take about 30 seconds to evaluate, but if you don't want to do the model computations yourself you can skip it.

In [12]:
#m.list_model_pts_to_run('./sim_list.h5')

The code in the next cell will actually do the model computations. On my two-core laptop, it takes about 24 minutes to evaluate. Assuming your processor supports multithreading (almost all modern ones do), you should set `n_jobs` to be twice the number of cores on your machine if you want to run this cell efficiently.

You can also just skip this cell and instead evaluate the following one to just load in the results of the computation that I did. :)

In [13]:
#sim_list = dd.io.load('./sim_list.h5')
#outputs=Parallel(n_jobs=4,verbose=7)(delayed(tefunnew_singlept)(sim[1][m.ec_names],sim[1][m.param_names]) for sim in sim_list.iterrows())
#sim_list['P'] = outputs
#dd.io.save('sim_outputs.h5',sim_list)

In [14]:
m.attach_model(mode='file',fpath='sim_outputs.h5')

On a sparse grid like this, it's important that the error values we use (i.e. standard deviation of Gaussians used for likelihood) are big enough to reach between boxes. This function computes the distance in output variable between model boxes at every experimental condition point and adds it to a column in model_data called 'deltas.'

In [15]:
m.calc_model_gradients()
m.model_data.sample(10)

,P0,fs,r,Z,n,T,R,P,deltas
26735,1.000000e-33,1.1,1.1,0.0,40.0,30.0,17.33330,1.351005e-44,7.267651e-22
239191,1.000000e-21,0.5,1.7,8.0,40.0,120.0,19.33330,4.033030e-44,4.711418e-21
64060,1.000000e-31,1.1,1.7,-8.0,40.0,140.0,9.66667,6.344674e-64,3.532274e-41
43970,1.000000e-31,-0.1,-0.7,-4.0,40.0,100.0,7.00000,3.368439e-01,8.703414e+00
154001,1.000000e-25,-0.1,-0.1,-4.0,40.0,10.0,12.00000,3.025768e-03,3.025768e-03
203883,1.000000e-23,1.1,-0.7,4.0,40.0,10.0,14.00000,9.238557e-03,1.289975e-02
126473,1.000000e-27,0.5,0.5,-4.0,40.0,20.0,14.66670,7.272151e-13,2.062945e-03
130021,1.000000e-27,0.5,1.7,4.0,40.0,60.0,5.33333,4.047108e-58,4.859070e-35
125845,1.000000e-27,0.5,-0.1,4.0,40.0,150.0,5.33333,2.061759e-01,9.566123e-01
49191,1.000000e-31,-0.1,1.1,8.0,40.0,100.0,8.00000,4.275323e-51,2.445266e-27


As you can see, because our grid is super sparse, the deltas are actually larger than the actual output values right now!

### First Bayes!
The `run` function randomizes the order of observations and stops feeding them in by default when 80% of the probability mass resides in 5% of the parameter space. These parameters can be tuned using the input parameters `th_pm` (default 0.8) and `th_pv` (default 0.05).

__If you don't want to have to run the new simulations yourself (they'll take longer than the first batch), don't run the code in this cell - I just left it so you can see what *was* run.__

(Because the `run` function randomizes observations, if you run it, the subdivided cells will likely not match exactly and you'll get an error if you try to just load in the results from my new simulation run)

In [16]:
#m.run()
#m.save_state(filename='states/state_1.h5')

Here we just load the model state that I saved and carry onward.

In [17]:
#m = bym.model(load_state=True, state_file='states/state_1.h5')
#%matplotlib inline
#m.probs.visualize()

I'm not sure what's going on with the upper left box right now. I'll fix it...

### Subdivide!
I've found that 0.001 seems to be a reasonable threshold probability for boxes to subdivide on the first round so that's the default value, but you can feed in other numbers for `threshold_prob` to this function.

Note that the `subdivide` function divides not only boxes meeting the threshold but any boxes immediately neighboring those. It will also write out an HDF5 of the new simulations that need to be run; that step can take awhile (this cell takes a few minutes on my computer) because it's writing every combination of new parameter points AND experimental condition points.

Again, if you don't want to run it, you can skip this cell and just use the `load_state` line in the next cell to start where I left off.

In [18]:
#m.subdivide()
#m.save_state('states/state_2.h5')

It's worth noting that there were originally 875 boxes in our super sparse grid, so in this case a majority of them were subdivided, which isn't too surprising.

### Run (more simulations and then) more inference!
I ran the batch of new simulations on Peregrine; the results are in the file `new_sim_outputs_.h5` which we'll load in here to do the next round of inference.

This cell takes about a minute to run.

In [19]:
#m = bym.model(load_state=True,state_file='states/state_2.h5')
#m.attach_model(mode='add',fpath='new_sim_outputs_1.h5')
#m.save_state('states/state_3.h5')

In [21]:
m = bym.model(load_state=True,state_file='states/state_3.h5')
#m.run()

In [ ]:
m.needs_new_model_data

In [ ]:
len(m.probs.points)

In [22]:
m.model_data[m.model_data['deltas'].isnull()]

,P,P0,R,T,Z,deltas,error,fs,n,r
0,2.003845e-06,3.162278e-34,1.00000,10.0,-9.0,NaN,NaN,-0.85,40.0,-0.85
1,3.998190e-06,3.162278e-34,2.00000,10.0,-9.0,NaN,NaN,-0.85,40.0,-0.85
2,5.983087e-06,3.162278e-34,3.00000,10.0,-9.0,NaN,NaN,-0.85,40.0,-0.85
3,7.958585e-06,3.162278e-34,4.00000,10.0,-9.0,NaN,NaN,-0.85,40.0,-0.85
4,9.924734e-06,3.162278e-34,5.00000,10.0,-9.0,NaN,NaN,-0.85,40.0,-0.85
5,1.188158e-05,3.162278e-34,6.00000,10.0,-9.0,NaN,NaN,-0.85,40.0,-0.85
6,1.382919e-05,3.162278e-34,7.00000,10.0,-9.0,NaN,NaN,-0.85,40.0,-0.85
7,1.576759e-05,3.162278e-34,8.00000,10.0,-9.0,NaN,NaN,-0.85,40.0,-0.85
8,1.769683e-05,3.162278e-34,9.00000,10.0,-9.0,NaN,NaN,-0.85,40.0,-0.85
9,1.961698e-05,3.162278e-34,10.00000,10.0,-9.0,NaN,NaN,-0.85,40.0,-0.85


In [ ]:
m.probs.points[m.probs.points['new']==True].sample(10)

In [ ]:
m.probs.find_neighbor_boxes(6596)

In [ ]:
all_vals = {param['name']:m.probs.all_current_values(param['name']) for param in m.probs.params}

In [ ]:
this_point = m.probs.points.iloc[6597]

In [ ]:
indices_to_intersect = []

In [ ]:
for param in [m.probs.params[0]]:
#for param in m.probs.params:
    this_param_val = this_point[param['name']]
    this_param_index = all_vals[param['name']].index(this_param_val)

    # handle the edge cases
    if not this_param_index == 0:
        down_delta = (all_vals[param['name']][this_param_index]-all_vals[param['name']][this_param_index-1])
    else:
        down_delta=0
    if not this_param_index == len(all_vals[param['name']])-1:
        up_delta = (all_vals[param['name']][this_param_index+1]-all_vals[param['name']][this_param_index])
    else:
        up_delta=0
    gt = m.probs.points[param['name']]>=this_param_val-down_delta
    lt = m.probs.points[param['name']]<=this_param_val+up_delta
    this_set = m.probs.points[gt & lt]
    indices_to_intersect.append(set(this_set.index.values))

In [ ]:
this_set.tail()

In [ ]:
indices_to_intersect=[]
for param in m.probs.params:
    this_param_val = this_point[param['name']]
    this_param_index = all_vals[param['name']].index(this_param_val)

    # handle the edge cases
    if not this_param_index == 0:
        down_delta = (all_vals[param['name']][this_param_index]-all_vals[param['name']][this_param_index-1])
    else:
        down_delta=0
    if not this_param_index == len(all_vals[param['name']])-1:
        up_delta = (all_vals[param['name']][this_param_index+1]-all_vals[param['name']][this_param_index])
    else:
        up_delta=0
    gt = m.probs.points[param['name']]>=this_param_val-down_delta
    lt = m.probs.points[param['name']]<=this_param_val+up_delta
    this_set = m.probs.points[gt & lt]
    indices_to_intersect.append(set(this_set.index.values))

In [ ]:
indices = list(set.intersection(*indices_to_intersect))

In [ ]:
len(indices)